In [332]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/youtube.csv",index_col=0)
df.head(5)

,title,channelTitle,categoryId,view_count,likes,dislikes,comment_count,channelId,trending_date2
0,[신병] 물자창고,장삐쭈,23,1893473,38249,730,8595,UChbE5OZQ6dRHECsX0tEPEZQ,2021-01-01
1,RAIN(비) - 나로 바꾸자 Switch to me (duet with JYP) MV,RAIN's Official Channel,10,2600864,0,0,20129,UCxXgIeE5hxWxHG6dz9Scg2w,2021-01-01
2,2020년 제야의 종 온라인 타종행사 | 보신각 현장 행사는 진행하지 않습니다.,서울시 · Seoul,29,347049,3564,120,178,UCZUPZW5idAxYp-Asj__lVAA,2021-01-01
3,고기남자의 칠면조 파티,고기남자 MeatMan,26,528458,15372,280,3470,UCT3CumbFIJiW33uq0UI3zlg,2021-01-01
4,골목 3mc를 분노하게 만든 마음고생이 심했을 공릉 백반집 사장님의 푸념?! [예능...,스브스밥집,24,494904,3918,111,3142,UCdWgRSfttvDucq4ApcCg5Mw,2021-01-01


In [333]:
# 인기동영상 제작횟수가 많은 채널 상위 10개명을 출력하라 (날짜기준, 중복포함)
top_channels = df["channelTitle"].value_counts(ascending=False).index.tolist() # value_counts로 인덱스로 들어간 채널명을 가져와 리스트로 변경
top_channels
# 논란으로 인기동영상이 된 케이스를 확인하고 싶다. dislikes수가 like 수보다 높은 동영상을 제작한 채널을 모두 출력하라
df.query('dislikes > likes')['channelTitle'].unique() # 쿼리 함수, 고유값찾기
# 채널명을 바꾼 케이스가 있는지 확인하고 싶다. channelId의 경우 고유값이므로 이를 통해 채널명을 한번이라도 바꾼 채널의 갯수를 구하여라
change = df.groupby('channelId')['channelTitle'].nunique() # 채널id별 채널명의 고유값 갯수 가져오고 채널id가 2개 이상인 값 추출, 그 값의 length
change_channel = change[change > 1]
len(change_channel)
# 일요일에 인기있었던 영상들중 가장많은 영상 종류(categoryId)는 무엇인가?
df['trending_date'] = pd.to_datetime(df['trending_date2'], format='%Y-%m-%d') # sql에 TO_DATE와 유사한 기능 yyyy면 y가 대문자
sunday_df = df[df['trending_date'].dt.weekday == 6] # dt.weekday()는 0이 월요일, 6이 일요일 tending_date가 dt.weekday 6 인 값들
sundday_categoryId = sunday_df['categoryId'].value_counts() # 6인 값들의 채널id들
sundday_categoryId.head(1)
# 각 요일별 인기 영상들의 categoryId는 각각 몇개 씩인지 하나의 데이터 프레임으로 표현하라
df['tranding_date'] = pd.to_datetime(df['trending_date2'], format='%Y-%m-%d') # pd.to_datetime으로 변환
df['day_name'] = df['tranding_date'].dt.day_name() # dt.day_name으로 요일이름 가져오기
group = df.groupby(['day_name', 'categoryId']).size().reset_index(name="count")# 요일이름 채널 아이디의 사이즈 출력하고 인덱스로 바꾼뒤 인덱스 초기화 후 count 컬럼 생성
pivot_table = group.pivot(index='day_name', columns='categoryId', values='count') # 피봇으로 인덱스,컬럼,값 지정
pivot_table.transpose() # 채널id와 요일이름 위치변경
# 댓글의 수로 (comment_count) 영상 반응에 대한 판단을 할 수 있다. viewcount대비 댓글수가 가장 높은 영상을 확인하라 (view_count값이 0인 경우는 제외한다)
zero = df.loc[df['view_count'] != 0] # 조회수가 0인 값 제외
comment = zero['comment_count'] / zero['view_count'] # 비율 계산 
best_index = comment.sort_values(ascending=False).index[0] # view카운트 대비 댓글수가 가장 높은 영상의 채널의 인덱스
result = df.loc[best_index, 'title'] # 해당 인덱스의 제목 출력
result
# 댓글의 수로 (comment_count) 영상 반응에 대한 판단을 할 수 있다.viewcount대비 댓글수가 가장 낮은 영상을 확인하라 (view_counts, ratio값이 0인경우는 제외한다.)
zero = df.loc[df['view_count'] != 0]
comment = zero['comment_count'] / zero['view_count']
ratio = comment[comment != 0] # 비율 계산 후 0이 아닌값
min_ratio = ratio.idxmin() # 0이 아닌 값중에 제일 낮은 값 idxmin() 인덱스 중 최소값
title = zero.loc[min_ratio, 'title'] 
title
# like 대비 dislike의 수가 가장 적은 영상은 무엇인가? (like, dislike 값이 0인경우는 제외한다)
zero = df.loc[(df['likes'] != 0) & (df['dislikes'] != 0),['likes', 'dislikes', 'title']] # loc의 and 연산 0제외
ratio = zero['dislikes'] / zero['likes'] # 비율 계산
min_likes = ratio.idxmin() # 인덱스 최소값
title = zero.loc[min_likes,'title'] # 인덱스 최소값 영상의 제목
title
# 가장많은 트렌드 영상을 제작한 채널의 이름은 무엇인가? (날짜기준, 중복포함)
best_create = df['channelId'].value_counts().index[0]  # 채널id가 가장 많은 인덱스
title = df.loc[df['channelId']==best_create,'channelTitle'].unique() # id가 많은, 영사잉 많은 채널 id의 채널 이름 , 중복제거
title
# 20회(20일)이상 인기동영상 리스트에 포함된 동영상의 숫자는?
best_video = (df[['title','channelTitle']].value_counts() >= 20).sum()
best_video

np.int64(40)

In [334]:
top_view= df['view_count'].head(10).sort_values(ascending=False).index

top_view_rows = df.loc[top_view]
display(top_view_rows)

,title,channelTitle,categoryId,view_count,likes,dislikes,comment_count,channelId,trending_date2,trending_date,tranding_date,day_name
1,RAIN(비) - 나로 바꾸자 Switch to me (duet with JYP) MV,RAIN's Official Channel,10,2600864,0,0,20129,UCxXgIeE5hxWxHG6dz9Scg2w,2021-01-01,2021-01-01,2021-01-01,Friday
7,[아이유의 팔레트] 팔레트에 '있지' (With ITZY) Ep.4,이지금 [IU Official],24,2420537,200301,956,14641,UC3SyT4_WLHzN7JmHQwKQZww,2021-01-01,2021-01-01,2021-01-01,Friday
0,[신병] 물자창고,장삐쭈,23,1893473,38249,730,8595,UChbE5OZQ6dRHECsX0tEPEZQ,2021-01-01,2021-01-01,2021-01-01,Friday
9,나 숙취 있어! 알딸딸해! 필름 끊겼어를 영어로 어떻게 할까요?,Sarah Kim,27,1358098,8224,820,1406,UCMXFwld5hCpMc2-49cndljA,2021-01-01,2021-01-01,2021-01-01,Friday
5,"(EN/CN/ID) 월드스타 한 분 더 모셨습니다. 비x박진영 & 싸이, 비주얼 가...",시즌비시즌 Season B Season,24,1274464,26660,368,2701,UCZf4ZESHAIuRtZ-eoMSL97A,2021-01-01,2021-01-01,2021-01-01,Friday
8,답답한 집콕.. 2020년은 예능으로 존버(존중하며 버팀)했다 MBC 연예대상 하이...,14F 일사에프,26,1001581,9462,337,1684,UCLKuglhGlMmDteQKoniENIQ,2021-01-01,2021-01-01,2021-01-01,Friday
6,[미스트롯2] 많은 참가자가 라이벌로 뽑은 명실상부 트롯 천재! 전유진이 부르는 '...,TV CHOSUN JOY,24,682608,7435,434,1943,UCYeeEwH_s9yD9HQe4e2JuBw,2021-01-01,2021-01-01,2021-01-01,Friday
3,고기남자의 칠면조 파티,고기남자 MeatMan,26,528458,15372,280,3470,UCT3CumbFIJiW33uq0UI3zlg,2021-01-01,2021-01-01,2021-01-01,Friday
4,골목 3mc를 분노하게 만든 마음고생이 심했을 공릉 백반집 사장님의 푸념?! [예능...,스브스밥집,24,494904,3918,111,3142,UCdWgRSfttvDucq4ApcCg5Mw,2021-01-01,2021-01-01,2021-01-01,Friday
2,2020년 제야의 종 온라인 타종행사 | 보신각 현장 행사는 진행하지 않습니다.,서울시 · Seoul,29,347049,3564,120,178,UCZUPZW5idAxYp-Asj__lVAA,2021-01-01,2021-01-01,2021-01-01,Friday


In [335]:
import pandas as pd


channel =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/channelInfo.csv')
video =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/videoInfo.csv')
display(channel.head())
display(video.head())

,channelid,subcnt,viewcnt,videocnt,ct,channelname
0,UCkQCwnkQfgSuPTTnw_Y7v7w,1310000,410238653,736,2021-09-30 03:01:03,꽈뚜룹
1,UCkQCwnkQfgSuPTTnw_Y7v7w,1310000,412531322,736,2021-09-30 09:01:03,꽈뚜룹
2,UCkQCwnkQfgSuPTTnw_Y7v7w,1310000,412531322,735,2021-09-30 15:01:03,꽈뚜룹
3,UCkQCwnkQfgSuPTTnw_Y7v7w,1310000,412531322,737,2021-09-30 21:01:03,꽈뚜룹
4,UCkQCwnkQfgSuPTTnw_Y7v7w,1320000,412531322,737,2021-10-01 03:01:04,꽈뚜룹


,videopk,viewcnt,likecnt,dislikecnt,favoritecnt,cmcnt,ct,videoname
0,c5JQp6xafqc,1667010,30474,706,0,6587,2021-10-10 15:20:03,공범 EP1
1,c5JQp6xafqc,1669089,30495,707,0,6589,2021-10-10 15:30:03,공범 EP1
2,c5JQp6xafqc,1674759,30522,711,0,6596,2021-10-10 15:40:02,공범 EP1
3,c5JQp6xafqc,1677026,30555,712,0,6604,2021-10-10 15:50:03,공범 EP1
4,c5JQp6xafqc,1681824,30585,713,0,6600,2021-10-10 16:00:03,공범 EP1


In [336]:
# 각 데이터의 ‘ct’컬럼을 시간으로 인식할수 있게 datatype을 변경하고 video 데이터의 videoname의 각 value 마다 몇개의 데이터씩 가지고 있는지 확인하라
channel['ct'] = pd.to_datetime(channel['ct']) # to_datetime
# channel['ct'].dtypes
video['videoname'].value_counts()
# 수집된 각 video의 가장 최신화 된 날짜의 viewcount값을 출력하라
ans2 = video[['viewcnt','videoname','ct']].sort_values(by='viewcnt', ascending=False)\
    .drop_duplicates(subset='videoname', keep='first')\
    .reset_index(drop=True) # 3컬럼을 가져와 viewcnt 기준으로 내림차순 정렬 후 videoname의 중복제거,첫번째 값만 남김 인덱스 초기화 후 인덱스 컬럼 삭제
ans2
# Channel 데이터중 2021-10-03일 이후 각 채널의 처음 기록 됐던 구독자 수(subcnt)를 출력하라
channel['ct'] = pd.to_datetime(channel['ct'])  # ct를 datetime으로 변환
channel_after = channel[channel['ct'] >= '2021-10-03'] # 2021-10-03 이후 데이터만 필터링
channel_sorted = channel_after.sort_values(by='ct') # ct 기준으로 오름차순 정렬 (먼저 수집된 게 위로)
channel_first = channel_sorted.drop_duplicates(subset='channelname', keep='first') # 채널마다 처음 나오는 값만 남김
ans3 = channel_first[['channelname', 'subcnt']].reset_index(drop=True) # 컬럼 정리
ans3
# 각채널의 2021-10-03 03:00:00 ~ 2021-11-01 15:00:00 까지 구독자수 (subcnt) 의 증가량을 구하여라
channel['ct'] = pd.to_datetime(channel['ct']) # 날짜 형식 변환
channel_range = channel[(channel['ct'] >= '2021-10-03 03:00:00') &  (channel['ct'] <= '2021-11-01 15:00:00')]  # 해당 날짜 범위로 필터링
channel_sorted = channel_range.sort_values(by='ct') # 시간순 정렬
grouped = channel_sorted.groupby('channelname')['subcnt'] # 채널별 구독자 수
first = grouped.first() # 제일 먼저 구한 값, 내림차순으로 맨 아래값 시간순으로 처음, 옛날 구독자 수
last = grouped.last() # 마지막 구독자 수, 최근 구독자 수
increase = last - first # 구독자 증가량 계산 (마지막 - 처음)
ans4 = pd.DataFrame({
    'channelname': increase.index,
    'del': increase.values
}).reset_index(drop=True)
ans4
# 각 비디오는 10분 간격으로 구독자수, 좋아요, 싫어요수, 댓글수가 수집된것으로 알려졌다. 공범 EP1의 비디오정보 데이터중 수집간격이 5분 이하, 20분이상인 데이터 구간( 해당 시점 전,후) 의 시각을 모두 출력하라
video['ct'] = pd.to_datetime(video['ct'])
ep1 = video[video['videoname'].str.strip() == '공범 EP1'] # 공범 EP1 가져옴
ep1 = ep1.sort_values(by='ct') # 시간 순 정렬
ep1['prev_ct'] = ep1['ct'].shift(1) # 이전 시간 만들기 (한 행씩 밀기)
ep1['gap_min'] = (ep1['ct'] - ep1['prev_ct']).dt.total_seconds() / 60 # 시간 차이 계산 (분 단위)
ans5 = ep1[(ep1['gap_min'] <= 5) | (ep1['gap_min'] >= 20)]# 5분 이하 또는 20분 이상인 구간만 추출
ans5 = ans5[video.columns]  # 원래 컬럼 순서로만 출력
ans5
# 각 에피소드의 시작날짜(년-월-일)를 에피소드 이름과 묶어 데이터 프레임으로 만들고 출력하라
video['ct'] = pd.to_datetime(video['ct'])
video_sorted = video.sort_values(by='ct') # 시간 순으로 정렬
video_first = video_sorted.drop_duplicates(subset='videoname', keep='first') # 에피소드 이름마다 가장 처음값만 남기기
video_first['date'] = video_first['ct'].dt.date # 날짜만 추출 (yyyy-mm-dd)
ans6 = video_first[['date', 'videoname']]
ans6
# “공범” 컨텐츠의 경우 19:00시에 공개 되는것으로 알려져있다. 공개된 날의 21시의 viewcnt, ct, videoname 으로 구성된 데이터 프레임을 viewcnt를 내림차순으로 정렬하여 출력하라
video['ct'] = pd.to_datetime(video['ct'])
video['hour'] = video['ct'].dt.hour # 시간에서 '시(hour)'만 추출
video_21 = video[(video['videoname'].str.contains('공범')) & (video['hour'] == 21)]\
.drop_duplicates(subset='videoname', keep='first') # '공범' 포함 + 21시에 수집된 데이터만 선택, videoname 중복제거
ans7 = video_21[['videoname', 'viewcnt', 'ct']]\
    .sort_values(by='viewcnt', ascending=False).reset_index(drop=True) # 필요한 컬럼만 정리하고 조회수 내림차순 정렬
ans7
# video 정보의 가장 최근 데이터들에서 각 에피소드의 싫어요/좋아요 비율을 ratio 컬럼으로 만들고 videoname, ratio로 구성된 데이터 프레임을 ratio를 오름차순으로 정렬하라
video['ct'] = pd.to_datetime(video['ct'])
video_sorted = video.sort_values(by='ct', ascending=False) # 시간 순서로 내림차순 정렬
video_latest = video_sorted.drop_duplicates(subset='videoname', keep='first') # 각 에피소드에서 가장 최근 데이터 하나씩만 남기기
video_latest['ratio'] = video_latest['dislikecnt'] / video_latest['likecnt'] # 비율 계산
ans8 = video_latest[['videoname', 'ratio']].sort_values(by='ratio').reset_index(drop=True) # 필요한 컬럼만 정리하고 오름차순 정렬
ans8
# 2021-11-01 00:00:00 ~ 15:00:00까지 각 에피소드별 viewcnt의 증가량을 데이터 프레임으로 만드시오
video['ct'] = pd.to_datetime(video['ct'])
video_range = video[(video['ct'] >= '2021-11-01 00:00:00') & (video['ct'] <= '2021-11-01 15:00:00')] # 원하는 시간 범위로 필터링
video_sorted = video_range.sort_values(by='ct') # 시간 순으로 정렬
grouped = video_sorted.groupby('videoname')['viewcnt'] # 이름별 조회수
first = grouped.first()
last = grouped.last()
increase = last - first # 증가량 계산
ans9 = pd.DataFrame({
    'viewcnt': increase
})
ans9
# video 데이터 중에서 중복되는 데이터가 존재한다. 중복되는 각 데이터의 시간대와 videoname 을 구하여라
video['ct'] = pd.to_datetime(video['ct'])
duplicates = video[video.duplicated()] # 완전히 같은 행이 중복된 것 찾기
ans10 = duplicates[['videoname', 'ct']]
ans10

C:\Users\pc\AppData\Local\Temp\ipykernel_6316\1181009091.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  video_first['date'] = video_first['ct'].dt.date # 날짜만 추출 (yyyy-mm-dd)
C:\Users\pc\AppData\Local\Temp\ipykernel_6316\1181009091.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  video_latest['ratio'] = video_latest['dislikecnt'] / video_latest['likecnt'] # 비율 계산


,videoname,ct
722,공범 EP1,2021-10-13 09:41:37
3927,공범 EP2,2021-10-13 09:41:37


In [337]:
import pandas as pd

df= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/worldcup/worldcupgoals.csv')
df

,Player,Goals,Years,Country
0,Miroslav Klose,16,2002-2006-2010-2014,Germany
1,Ronaldo,15,1998-2002-2006,Brazil
2,Gerd Muller,14,1970-1974,Germany
3,Just Fontaine,13,1958,France
4,Pele,12,1958-1962-1966-1970,Brazil
...,...,...,...,...
1290,Josip Skoblar,1,1962,Yugoslavia
1291,Safet Susic,1,1982-1990,Yugoslavia
1292,Aleksandar Tirnanic,1,1930,Yugoslavia
1293,Djordje Vujadinovic,1,1930,Yugoslavia


In [338]:
# 주어진 전체 기간의 각 나라별 골득점수 상위 5개 국가와 그 득점수를 데이터프레임형태로 출력하라
ans1 = df['Goals'] = df['Goals'].fillna(0)
# df['Goals'].isna().sum()
goals = df.groupby('Country')['Goals'].sum()
ans2 = goals.sort_values(ascending=False).head(5).to_frame() # 득점 수 기준으로 내림차순 정렬 후 상위 5개 추출, DataFrame형식변환
ans2
# 주어진 전체기간동안 골득점을 한 선수가 가장 많은 나라 상위 5개 국가와 그 선수 숫자를 데이터 프레임 형식으로 출력하라
goals = df[df['Goals'] > 0] # 먼저 Goals > 0인 데이터만 추춣
goals_country = goals.groupby('Country')['Player'].nunique() # 나라별로 득점한 선수 수 세기 선수 중복 제거
ans3 = goals_country.sort_values(ascending=False).head(5).to_frame() # 내림차순 정렬 후 상위 5개 추출, DataFrame형식변환
ans3
# Years 컬럼은 년도 -년도 형식으로 구성되어있고, 각 년도는 4자리 숫자이다. 년도 표기가 4자리 숫자로 안된 케이스가 존재한다. 해당 건은 몇건인지 출력하라
df['years'] = df['Years'].str.split('-')
df['find'] = df['years'].apply(lambda x: any((not y.isdigit()) or (len(y) != 4) for y in x)) # split으로 나눈 year가 4자리가 아닌 값
# any() 안에 하나라도 true가 있으면 true 반환 str.isdigit() 문자가 하나라도 있으면 False 반환, 공백,마이너스,마침표가 들어가도 False, 여기선 not y니까 True
ans4 = df['find'].sum()
ans4

# **Q3에서 발생한 예외 케이스를 제외한 데이터프레임을 df2라고 정의하고 데이터의 행의 숫자를 출력하라 (아래 문제부터는 df2로 풀이하겠습니다) **
df2 = df[df['find']==False].reset_index(drop=True)  # Q3에서 발생한 예외 케이스를 제외하고 인덱스 초기화
df2.shape[0]  # 행의 숫자 출력
# 월드컵 출전횟수를 나타내는 ‘LenCup’ 컬럼을 추가하고 4회 출전한 선수의 숫자를 구하여라
df2['LenCup'] = df2['Years'].apply(lambda x: len(x.split('-')))  # 'Years' 컬럼을 '-'로 나누어 출전 횟수 계산
ans5 = df2[df2['LenCup'] == 4].shape[0]  # 4회 출전한 선수의 숫자
ans5
# Yugoslavia 국가의 월드컵 출전횟수가 2회인 선수들의 숫자를 구하여라
df2['LenCup'] = df2['Years'].apply(lambda x: len(x.split('-')))
cunt_cnt = df2[(df2['Country'] == 'Yugoslavia') & (df2['LenCup'] == 2)]  # Yugoslavia 국가의 월드컵 출전횟수가 2회인 선수들 필터링
cnt = cunt_cnt['Player'].nunique()  # 중복 제거 후 선수 수 세기
ans6 = cnt
ans6
# 2002년도에 출전한 전체 선수는 몇명인가?
years = df2[df2['Years'].str.contains('2002')]['Player'].nunique() # dr2의 Years 컬럼 선택,문자열 변환,2002포함 여부,2002년의 선수숫자 중복제거 숫자 추출
ans7 = years
ans8
# 이름에 ‘carlos’ 단어가 들어가는 선수의 숫자는 몇 명인가? (대, 소문자 구분 x)
ans8 = df2[df2['Player'].str.contains('carlos',case=False,na=False)] # 선수이름에 carlos 대소문자 구분 없이 포함 여부 True값만 반환
len(ans8) #True의 레코드 수 출력
# 월드컵 출전 횟수가 1회뿐인 선수들 중에서 가장 많은 득점을 올렸던 선수는 누구인가?
df2['Lencup'] = df2['Years'].apply(lambda x: len(x.split('-')))
players = df2[(df2['Lencup']==1)] # 출전횟수가 1인 선수들
max_goales = players['Goals'].max() # goal횟수가 제일 많은 사람
max_goal_p = players[players['Goals']==max_goales] # 출전 횟수가 1인 선수중 goal 횟수가 제일 많은 사람
ans9 = max_goal_p
ans9['Player']
# 월드컵 출전횟수가 1회 뿐인 선수들이 가장 많은 국가는 어디인가?
df2['Lencup'] = df2['Years'].apply(lambda x: len(x.split('-')))
ans10 = df2[df2['Lencup']==1]['Country'].value_counts().idxmax() # 출전 횟수가 1인 사람 출전 횟수가 1인 사람의국가
ans10
# 출전 횟수가 1인 사람의 국가 중 제일 높은 값 여기서 value_counts()는 국가 이름이 인덱스, 출전 횟수가 값으로 들어감

'Brazil'

In [339]:
import pandas as pd
df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bicycle/seoul_bi.csv')
df.head()

,대여일자,대여시간,대여소번호,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리,사용시간
0,2021-06-01,0,3541,정기권,F,~10대,1,0.00,0.00,0.00,8
1,2021-06-01,0,765,정기권,F,~10대,1,27.21,0.35,1526.81,19
2,2021-06-01,0,2637,정기권,F,~10대,1,41.40,0.37,1608.56,18
3,2021-06-01,0,2919,정기권,F,~10대,1,0.00,0.00,0.00,75
4,2021-06-01,0,549,정기권,F,~10대,1,13.04,0.17,731.55,6


In [340]:
# 대여일자별 데이터의 수를 데이터프레임으로 출력하고, 가장 많은 데이터가 있는 날짜를 출력하라
# frame = df['대여일자'].value_counts().sort_index().to_frame(name='count') # 대여일자 별 데이터 추출 인덱스로(대영리자) 정렬 데이터프레임화,
# data_max = frame['count'].idxmax() # count의 제일 높은 값
# display(frame)
# data_max
# 각 일자의 요일을 표기하고 (‘Monday’ ~’Sunday’) ‘day_name’컬럼을 추가하고 이를 이용하여 각 요일별 이용 횟수의 총합을 데이터 프레임으로 출력하라
# df['대여일자'] = pd.to_datetime(df['대여일자']) # 타입 변경
# df['day_name'] = df['대여일자'].dt.day_name() # 요일별 이름 생성
# week_cnt = df['day_name'].value_counts() # 요일별 총합
# # week_cnt = df.groupby('day_name')['이용건수'].sum()
# ans2 = week_cnt.to_frame() # 데이터프레임화
# ans2
# 각 요일별 가장 많이 이용한 대여소의 이용횟수와 대여소 번호를 데이터 프레임으로 출력하라
# df['대여일자'] = pd.to_datetime(df['대여일자']) # 타입 변경
# df['day_name'] = df['대여일자'].dt.day_name() # 요일별 이름 생성
# rental_cnt = df.groupby(['day_name','대여소번호']).size().to_frame(name='size').sort_values(['day_name','size'],ascending=False).reset_index()
# # day_name, 대여소번호별로 그룹화 후 size로 사이즈 구하고 정렬 후 인덱스 초기화
# week_rental = rental_cnt.drop_duplicates(subset='day_name',keep='first').reset_index(drop=True)
# # 요일별 대여소 번호별 이용횟수 구하고 정렬 후 인덱스 초기화, 요일별로 중복제거
# ans3 = week_rental
# ans3
# 나이대별 대여구분 코드의 (일일권/전체횟수) 비율을 구한 후 가장 높은 비율을 가지는 나이대를 확인하라. 일일권의 경우 일일권 과 일일권(비회원)을 모두 포함하라
# day_Passes = df[df['대여구분코드'].str.contains('일일권',na=False)] # 일일권,일일권(비회원) 추출
# age = day_Passes.groupby('연령대코드').size() # 연령대별 일일권 사이즈 추출
# Passes = df.groupby('연령대코드').size() # 전체 연령대별 사이즈 추출
# ratio = (age / Passes).fillna(0) # 일일권 연령대 / 전체 연령대 비율 
# ratio_max = ratio.idxmax() # 연령대 비율중 제일 높은 값 추출
# ans4 = ratio.sort_values(ascending=False)# 값 내림차순 정렬
# display(ans4)
# ratio_max
# ratio = day_Passes['대여구분코드'] / Passes['대여구분코드'] # 비율 구하기
# ans4 = ratio['연령대코드']
# ans4['연령대코드']
# 연령대별 평균 이동거리를 구하여라
# age = df.groupby('연령대코드').size()
# Distance = df.groupby('연령대코드')['이동거리'].mean()
# ans5 = Distance
# ans5
# 연령대 코드가 20대인 데이터를 추출하고,이동거리값이 추출한 데이터의 이동거리값의 평균 이상인 데이터를 추출한다.최종 추출된 데이터를 대여일자, 
# 대여소 번호 순서로 내림차순 정렬 후 1행부터 200행까지의 탄소량의 평균을 소숫점 3째 자리까지 구하여라
# age = df[df['연령대코드']=='20대'] # 20대인 행만 필터링
# age_distance = age['이동거리'].mean() # 20대의 이동거리 평균 필터링
# avg_grate = age[age['이동거리']>=age_distance] # 20대의 이동거리 평균 이상값 필터링
# data = avg_grate.sort_values(by=['대여일자','대여소번호'],ascending=False) #  평균이상값을 대여일자,대여소 번호 기준으로 내림차순 정렬
# ans6 = pd.to_numeric(data.iloc[0:200]['탄소량']).mean() # 탄소량 데이타 타입변환
# round(ans6,3)
# 6월 7일 ~10대의 “이용건수”의 중앙값은?
# df['대여일자'] = pd.to_datetime(df['대여일자']) # 대여일자 데이트타입으로 변환
# tartget_day = df[(df['대여일자']=='2021-06-07') & (df['연령대코드'].str.contains('10대',na=False))] # 2021-06-07 추출 10대 포함 값 추출
# ans7 = tartget_day['이용건수'].median()
# ans7
# 평일 (월~금) 출근 시간대(오전 6,7,8시)의 대여소별 이용 횟수를 구해서 데이터 프레임 형태로 표현한 후 각 대여시간별 이용 횟수의 상위 3개 대여소와 이용횟수를 출력하라
# df['대여일자'] = pd.to_datetime(df['대여일자']) # 데이트타임으로 변환 
# df['요일'] = df['대여일자'].dt.weekday # 0 = 월, 6 = 일
# target_day = df[(df['요일'].between(0,4)) & (df['대여시간'].isin([6,7,8]))] # 요일의 0~4 월~금 and 대여시간 678 추출
# data = target_day.groupby(['대여시간', '대여소번호']).size().to_frame(name='이용횟수') # groupby 대여시간 대여소번호 별 이용횟수
# ans8 = data.groupby(level=0,as_index=False).apply(lambda x: x.nlargest(3, '이용횟수')) # 대여시간별 상위 3개 추출
# ans8.sort_index(ascending=False) # 내림차순 정렬
# 이동거리의 평균 이상의 이동거리 값을 가지는 데이터를 추출하여 추출데이터의 이동거리의 표본표준편차 값을 구하여라
# mean_distens = df[df['이동거리'] >= df['이동거리'].mean()] # 평균 이상 이동거리 추출
# ans9 = mean_distens['이동거리'].std()
# ans9
# 남성(‘M’ or ‘m’)과 여성(‘F’ or ‘f’)의 이동거리값의 평균값을 구하여라
mean_distens = df[df['성별'].isin(['M','F'])].groupby('성별')['이동거리'].mean()
ans10 = mean_distens
ans10

성별
F    3468.751197
M    3209.392786
Name: 이동거리, dtype: float64

In [341]:
import pandas as pd
df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/happy2/happiness.csv',encoding='utf-8')
df.head()

,행복랭킹,나라명,점수,상대GDP,사회적지원,행복기대치,선택의 자유도,관대함,부패에 대한인식,년도
0,1,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393,2019
1,2,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410,2019
2,3,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341,2019
3,4,Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118,2019
4,5,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298,2019


In [342]:
# 데이터는 2018년도와 2019년도의 전세계 행복 지수를 표현한다. 각년도의 행복랭킹 10위를 차지한 나라의 행복점수의 평균을 구하여라
# mean_happy = df[df['행복랭킹']==10]['점수'].mean() # 행복랭킹 10위의 잠수의 평균
# ans1 = mean_happy
# ans1
# 데이터는 2018년도와 2019년도의 전세계 행복 지수를 표현한다. 각년도의 행복랭킹 50위이내의 나라들의 각각의 행복점수 평균을 데이터프레임으로 표시하라
# mean_happy = df[df['행복랭킹']<=50].groupby('년도')['점수'].mean().to_frame() # 행복 랭킹 50위 이내 각 년도 별 점수 평균
# ans2 = mean_happy
# ans2
# 2018년도 데이터들만 추출하여 행복점수와 부패에 대한 인식에 대한 상관계수를 구하여라
# target_ysers = df[df['년도']==2018][['점수','부패에 대한인식']].corr().reset_index(drop=True) # 2018년도의 행복점수 와 부패에 대한인식 의 상관관계
# ans3 = target_ysers
# ans3
# 2018년도와 2019년도의 행복랭킹이 변화하지 않은 나라명의 수를 구하여라
# pivot = df.pivot(index='나라명', columns='년도', values='행복랭킹') # 피봇 테이블로 만들어 비교
# unchanged = (pivot[2018] == pivot[2019]).sum() # 값이 똑같다면 True, True의 갯수
# ans4 = unchanged
# ans4
# 2019년도 데이터들만 추출하여 각변수간 상관계수를 구하고 내림차순으로 정렬한 후 상위 5개를 데이터 프레임으로 출력하라. 컬럼명은 v1,v2,corr으로 표시하라
# zz = df[df['년도']==2019].select_dtypes(include='number').corr().unstack().to_frame().reset_index().dropna()
# result = zz[zz[0] !=1].sort_values(0,ascending=False).drop_duplicates(0)
# ans5 = result.head(5).reset_index(drop=True)
# ans5.columns = ['v1','v2','corr']
# display(ans5)
# 각 년도별 하위 행복점수의 하위 5개 국가의 평균 행복점수를 구하여라
# tail = df.groupby('년도').apply(lambda x: x.nsmallest(5, '점수')).reset_index(drop=True) # 각 년도의 하위 5개 점수 추출
# ans6 = tail.groupby('년도')['점수'].mean().reset_index() # 각 년도별 평균 행복점수 계산
# ans6
# 2019년 데이터를 추출하고 해당데이터의 상대 GDP 평균 이상의 나라들과 평균 이하의 나라들의 행복점수 평균을 각각 구하고 그 차이값을 출력하라
# target_year = df[df['년도']==2019]
# mean_gdp_up = target_year[target_year['상대GDP'] >= target_year['상대GDP'].mean()]['점수'].mean()
# # 2019년의 gdp 평균 이상
# mean_gdp_down = target_year[target_year['상대GDP'] <= target_year['상대GDP'].mean()]['점수'].mean()
# # 2019년의 gdp 평균 이하
# diff_gdp =   mean_gdp_up - mean_gdp_down
# ans7 = diff_gdp
# ans7
# 각년도의 부패에 대한인식을 내림차순 정렬했을때 상위 20개 국가의 부패에 대한인식의 평균을 구하여라
# ans8 = df.sort_values(['년도', '부패에 대한인식'], ascending=[True, False])\
#         .groupby('년도')\
#         .head(20)\
#         .groupby('년도')['부패에 대한인식']\
#         .mean() 
# ans8
# 2018년도 행복랭킹 50위 이내에 포함됐다가 2019년 50위 밖으로 밀려난 국가의 숫자를 구하여라
# after_cunt = set(df[(df['년도']==2018) & (df['행복랭킹']<=50)]['나라명']) # set() 함수 차집합 연산이 가능하게 해줌
# before_cunt = set(df[(df['년도']==2019) & (df['행복랭킹']<=50)]['나라명']) 
# ans9 = after_cunt - before_cunt # 2018년 에는 있었지만 2019년엔 없는 나라명
# len(ans9) # 뽑은 나라명의 길이
# 2018년,2019년 모두 기록이 있는 나라들 중 년도별 행복점수가 가장 증가한 나라와 그 증가 수치는?
# target_2018 = df[df['년도']==2018][['나라명','점수']]
# target_2019 = df[df['년도']==2019][['나라명','점수']] # 각 년도별 나라명,점수 추출
# merged = pd.merge(left=target_2018,right=target_2019,on='나라명',suffixes=('2018','2019'))  # merge 가 defalt innerjoin이라 2018,2019 합친 테이블
# merged['증가량'] = merged['점수2019'] - merged['점수2018'] # 증가량 계산
# max_cunt = merged.sort_values('증가량', ascending=False).iloc[0] # 증가량의 내림차순 정렬 후 첫번재 행 추출
# max_cunt[['나라명','증가량']]
# pivot = df.pivot(index='나라명',columns='년도',values='점수') # 피봇 테이블
# pivot = pivot.dropna(subset=[2018,2019])  # 컬럼에 결측값 제거
# pivot['증가량'] = pivot[2019] - pivot[2018] # 증가량 계산
# max_value = pivot['증가량'].max() # 증가량중에 제일 높은 값
# max_cunt = pivot['증가량'].idxmax() # 지금 index가 나라니까 idxmax() 증가량이 제일 높은 나라

# print(max_cunt)
# max_value

In [343]:
import pandas as pd
df= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/consum/Tetuan%20City%20power%20consumption.csv')
df.head()

,DateTime,Temperature,Humidity,Wind Speed,general diffuse flows,diffuse flows,Zone 1 Power Consumption,Zone 2 Power Consumption,Zone 3 Power Consumption
0,1/1/2017 0:00,6.559,73.8,0.083,0.051,0.119,34055.69620,16128.87538,20240.96386
1,1/1/2017 0:10,6.414,74.5,0.083,0.070,0.085,29814.68354,19375.07599,20131.08434
2,1/1/2017 0:20,6.313,74.5,0.080,0.062,0.100,29128.10127,19006.68693,19668.43373
3,1/1/2017 0:30,6.121,75.0,0.083,0.091,0.096,28228.86076,18361.09422,18899.27711
4,1/1/2017 0:40,5.921,75.7,0.081,0.048,0.085,27335.69620,17872.34043,18442.40964


In [344]:
# DateTime컬럼을 통해 각 월별로 몇개의 데이터가 있는지 데이터 프레임으로 구하여라
# df['Month'] = df['DateTime'].dt.month # 월만 추출
# ans1 = df['Month'].value_counts().sort_index().to_frame() # 원별 데이터 추출,인덱스 정렬, 데이터프레임화
# ans1
# 3월달의 각 시간대별 온도의 평균들 중 가장 낮은 시간대의 온도를 출력하라
# df['DateTime'] = pd.to_datetime(df['DateTime'])
# df['Month'] = df['DateTime'].dt.month # 월 추출
# targer_month = df[df['Month']==3] # 3원달 필터링
# df['Hour'] = df['DateTime'].dt.hour # 시간 추출
# # target_month = df[(df['Month']==3) & (df['Hour'])]['Temperature'].mean()
# Temperature_mean = targer_month.groupby('Hour')['Temperature'].mean() # 3월달의 시간 별 온도 평균 
# ans2 = Temperature_mean.sort_values().iloc[0] # 3월시간별온도평균의 오름차순 정렬 후 첫번째 값 추출
# ans2
# 3월달의 각 시간대별 온도의 평균들 중 가장 높은 시간대의 온도를 출력하라
# df['DateTime'] = pd.to_datetime(df['DateTime'])
# df['Month'] = df['DateTime'].dt.month # 월 추출
# targer_month = df[df['Month']==3] # 3원달 필터링
# df['Hour'] = df['DateTime'].dt.hour # 시간 추출
# Temperature_mean = targer_month.groupby('Hour')['Temperature'].mean() # 3월달의 시간 별 온도 평균 
# ans3 = Temperature_mean.sort_values(ascending=False).iloc[0] # 3월시간별온도평균의 내림차순 정렬 후 첫번째 값 추출
# ans3
# Zone 1 Power Consumption 컬럼의 value값의 크기가 Zone 2  Power Consumption 컬럼의 value값의 크기보다 큰 데이터들의 Humidity의 평균을 구하여라
# over = ((df['Zone 1 Power Consumption'].values) >= (df['Zone 2  Power Consumption'].values))
# over = df[df['Zone 1 Power Consumption'] > df['Zone 2  Power Consumption']]['Humidity'].mean()
# ans4 = over
# ans4
# 각 zone의 에너지 소비량의 상관관계를 구해서 데이터 프레임으로 표기하라
# ans5 = df.iloc[:,-3:].corr()
# ans5
# Temperature의 값이 10미만의 경우 A, 10이상 20미만의 경우 B,20이상 30미만의 경우 C, 그 외의 경우 D라고 할때 각 단계의 데이터 숫자를 구하여라
# step = df['Temperature'].apply(
#                                 lambda x: 'A' if x < 10 else
#                                         'B' if x < 20 else
#                                         'C' if x < 30 else
#                                         'D'
#                             )
# df['Temperature_category'] = step
# ans6 = df['Temperature_category'].value_counts()
# ans6.sort_index()
# 6월 데이터중 12시의 Temperature의 표준편차를 구하여라
# df['DateTime'] = pd.to_datetime(df['DateTime'])
# df['Month'] = df['DateTime'].dt.month
# df['Hour'] = df['DateTime'].dt.hour
# month_6 = df[(df['Month']==6) & (df['Hour']==12)]['Temperature'].std() # 6월,12시의 온도 표준편차
# ans7 = month_6
# ans7
# 6월 데이터중 12시의 Temperature의 분산을 구하여라
# df['DateTime'] = pd.to_datetime(df['DateTime'])
# df['Month'] = df['DateTime'].dt.month
# df['Hour'] = df['DateTime'].dt.hour
# month_6 = df[(df['Month']==6) & (df['Hour']==12)]['Temperature'].var() # 6월,12시의 온도 분산
# ans8 = month_6
# ans8
# Temperature의 평균이상의 Temperature의 값을 가지는 데이터를 Temperature를 기준으로 정렬 했을때 4번째 행의 Humidity 값은?
# mean_Temperature = df[df['Temperature']>=df['Temperature'].mean()] # 평균이상의 온도 필터링
# ans9 = mean_Temperature.sort_values(by='Temperature').iloc[3] # 온도 기준 정렬, 3번째 행 추출
# ans9['Humidity'] # 3번째 행의 Humidity
# **Temperature의 중간값 이상의 Temperature의 값을 가지는 데이터를Temperature를 기준으로 정렬 했을때 4번째 행의 Humidity 값은? **
# Temperature_med = df[df['Temperature']>=df['Temperature'].median()] # 중앙값 이상값 추출
# ans10 = Temperature_med.sort_values(by='Temperature').iloc[4] # 온도 기준 정렬 , 4번째 행 추출
# ans10['Humidity'] # 4번째 행의 Humidity
# result = df[df.Temperature >= df.Temperature.median()].sort_values('Temperature').Humidity.values[4]
# print(result)

In [345]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/pok/Pokemon.csv')
df.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [ ]:
# Legendary 컬럼은 전설포켓몬 유무를 나타낸다.전설포켓몬과 그렇지 않은 포켓몬들의 HP평균의 차이를 구하여라
# legend_hp_mean = df[df['Legendary']==True]['HP'].mean() 
# normal_hp_mean = df[df['Legendary']==False]['HP'].mean() # 레전더리가 값의 hp의 평균
# diff = legend_hp_mean - normal_hp_mean # 차이값 계산
# ans1 = diff
# ans1
# Type 1은 주속성 Type 2 는 부속성을 나타낸다. 가장 많은 부속성 종류는 무엇인가?
# ans2 = df['Type 2'].value_counts().idxmax() # Type 2 의 제일 값이 많은 인덱스
# ans2
# 가장 많은 Type 1 의 종의 평균 Attack 을 평균 Defense로 나눈값은?
# max_type1 = df['Type 1'].value_counts().index[0] # 가장많은 type1 필터링
# diff_tyoe1 = df[df['Type 1']==max_type1]['Attack'].mean() / df[df['Type 1']==max_type1]['Defense'].mean() # 나눈값 계산
# ans3 = diff_tyoe1
# ans3
# mean_att_type1 = max_type1['Type 1']['Attack'].mean()
# 포켓몬 세대(Generation) 중 가장많은 Legendary를 보유한 세대는 몇세대인가?
# Generation = df[df['Legendary']==True]['Generation'].value_counts() # 레전더리인 세대의 값
# ans4 = Generation.idxmax() # 제일 높은 값의 인덱스
# ans4
# ‘HP’, ‘Attack’, ‘Defense’, ‘Sp. Atk’, ‘Sp. Def’, ‘Speed’ 간의 상관 계수중 가장 절댓값이 큰 두 변수와 그 값을 구하여라
# corr = df.iloc[:,5:10].corr()
# corr_unstacked = corr.unstack()
# corr_filtered = corr_unstacked[corr_unstacked.index.get_level_values(0) != corr_unstacked.index.get_level_values(1)]
# corr_sorted = corr_filtered.reindex(corr_filtered.abs().sort_values(ascending=False).index)
# top_pair = corr_sorted.index[0]
# top_value = corr_sorted.iloc[0]
# print(top_pair,top_value) # ans5
# 각 Generation의 Attack으로 오름차순 정렬시 상위 3개 데이터들(18개)의 Attack의 전체 평균을 구하여라
# ans6 = df.sort_values(['Generation','Attack']).groupby('Generation').head(3)['Attack'].mean() 
# ans6
# 각 Generation의 Attack으로 내림차순 정렬시 상위 5개 데이터들(30개)의 Attack의 전체 평균을 구하여라
# ans7 = df.sort_values(['Generation','Attack'],ascending=[True,False]).groupby('Generation').head(5)['Attack'].mean() 
# ans7
# 가장 흔하게 발견되는 (Type1 , Type2) 의 쌍은 무엇인가?
# ans8 = df[['Type 1','Type 2']].value_counts(ascending=False).index[0]
# ans8
# 한번씩만 존재하는 (Type1 , Type2)의 쌍의 갯수는 몇개인가?
# tween = df[['Type 1','Type 2']].value_counts() # 쌍의 값
# ans9 = tween[tween==1].sum() # 쌍이 1개인 걸들의 합
# ans9
# 한번씩만 존재하는 (Type1 , Type2)의 쌍을 각 세대(Generation)은 각각 몇개씩 가지고 있는가? *
target = df[['Type 1','Type 2']].value_counts()
target2 =target[target==1]

lst = []
for value in target2.reset_index().values:
    t1 = value[0]
    t2 = value[1]
    
    sp = df[(df['Type 1']==t1) & (df['Type 2']==t2)]
    lst.append(sp)

ans10 = pd.concat(lst).reset_index(drop=True).Generation.value_counts().sort_index()
ans10



Generation
1     1
2     4
3     5
4    13
5     7
6     9
Name: count, dtype: int64

In [355]:
import pandas as pd
df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/body/body.csv')
df.head()

,측정나이,측정회원성별,신장 : cm,체중 : kg,체지방율 : %,이완기혈압(최저) : mmHg,수축기혈압(최고) : mmHg,악력D : kg,앉아윗몸앞으로굽히기 : cm,교차윗몸일으키기 : 회,제자리 멀리뛰기 : cm,등급
0,59.0,M,175.0,70.6,19.2,91.0,150.0,40.6,12.2,30.0,179.0,C
1,40.0,F,161.4,45.8,21.7,62.0,119.0,22.1,16.0,32.0,165.0,B
2,27.0,M,176.6,84.8,19.2,87.0,153.0,45.9,13.2,61.0,216.0,B
3,38.0,M,167.6,76.7,24.7,63.0,132.0,43.6,16.0,45.0,231.0,A
4,21.0,M,165.2,66.2,21.5,83.0,106.0,33.5,10.6,46.0,198.0,C


In [ ]:
# 전체데이터의 수축기혈압(최고) - 이완기혈압(최저)의 평균을 구하여라
# ans1 = (df['수축기혈압(최고) : mmHg'] - (df['이완기혈압(최저) : mmHg'])).mean()
# ans1
# 50~59세의 신장평균을 구하여라
# age_50 = df[(df['측정나이']>=50) & (df['측정나이']<60)] # 50대 필터링
# ans2 = age_50['신장 : cm'].mean() # 50대의 신장 평귱
# ans2
# 연령대 (20~29 : 20대 …) 별  인원수를 구하여라
# Age  = df['측정나이'].apply(lambda x: 
#                             '20대' if 20 <= x < 30 else
#                             '30대' if 30 <= x < 40 else
#                             '40대' if 40 <= x < 50 else 
#                             '50대' if 50 <= x < 60 else
#                             '60대')
# ans3 = Age.value_counts()
# ans3
# 연령대 (20~29 : 20대 …) 별 등급의 숫자를 데이터 프레임으로 표현하라
# df['연령대']  = df['측정나이'].apply(lambda x: 
#                             '20대' if 20 <= x < 30 else
#                             '30대' if 30 <= x < 40 else
#                             '40대' if 40 <= x < 50 else 
#                             '50대' if 50 <= x < 60 else
#                             '60대')
# ans4 = df[['연령대','등급']].sort_values('연령대').value_counts().to_frame('size').reset_index()
# ans4
# 남성 중 A등급과 D등급의 체지방률 평균의 차이(큰 값에서 작은 값의 차)를 구하여라
# Aman_mean = df[(df['측정회원성별']=='M') & (df['등급']=='A')]['체지방율 : %'].mean() 
# Dman_mean = df[(df['측정회원성별']=='M') & (df['등급']=='D')]['체지방율 : %'].mean() # 남자 등급별 체지방률 평균
# diff_rank = Dman_mean - Aman_mean # 차이값 계산
# ans5 = diff_rank
# ans5
# 여성 중 A등급과 D등급의 체중의 평균의 차이(큰 값에서 작은 값의 차)를 구하여라
# Awoman_mean = df[(df['측정회원성별']=='F') & (df['등급']=='A')]['체중 : kg'].mean() 
# Dwoman_mean = df[(df['측정회원성별']=='F') & (df['등급']=='D')]['체중 : kg'].mean()  # 여자 등급별 체중 평균
# diff_rank = Dwoman_mean - Awoman_mean # 차이값 계산
# ans6 = diff_rank
# ans6
# bmi는 자신의 몸무게(kg)를 키의 제곱(m)으로 나눈값이다. 데이터의 bmi 를 구한 새로운 컬럼을 만들고 남성의 bmi 평균을 구하여라
# df['bmi'] = df['체중 : kg'] / (df['신장 : cm']/100)**2 # 체중 / 신장 cm이니까 /100 하고 제곱
# ans7 = df[df['측정회원성별']=='M']['bmi'].mean() # 남성의 bmi 평균
# ans7
# bmi보다 체지방율이 높은 사람들의 체중평균을 구하여라
# fat = df[df['bmi'] < df['체지방율 : %']]['체중 : kg'].mean() # 쉽쥬
# ans8 = fat
# ans8
# 남성과 여성의 악력 평균의 차이를 구하여라
# a_man = df[df['측정회원성별']=='M']['악력D : kg'].mean()
# a_woman = df[df['측정회원성별']=='F']['악력D : kg'].mean() # 쉽쥬
# diff_mean = a_man - a_woman
# ans9 = diff_mean
# ans9
# 남성과 여성의 교차윗몸일으키기 횟수의 평균의 차이를 구하여라
# a_man = df[df['측정회원성별']=='M']['교차윗몸일으키기 : 회'].mean()
# a_woman = df[df['측정회원성별']=='F']['교차윗몸일으키기 : 회'].mean() # 쉽쥬
# diff_mean = a_man - a_woman
# ans10 = diff_mean
# ans10

np.float64(14.243156833157627)

In [411]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/weather/weather2.csv")
df.head()

,time,이화동강수,이화동기온,수영동강수,수영동기온
0,2020-01-01 00:00:00,0.1,-3.9,0.0,2.5
1,2020-01-01 01:00:00,0.0,-3.1,0.0,3.4
2,2020-01-01 02:00:00,0.0,-1.8,0.0,3.8
3,2020-01-01 03:00:00,0.0,-0.8,0.0,4.6
4,2020-01-01 04:00:00,0.0,-0.1,0.0,5.1


In [484]:
# 여름철(6월,7월,8월) 이화동이 수영동보다 높은 기온을 가진 시간대는 몇개인가?
# df['time'] = pd.to_datetime(df['time'])
# df['month'] = df['time'].dt.month
# df['hour'] = df['time'].dt.hour
# 월 = df[df['month'].isin([6,7,8])] # 6,7,8 월 추출
# 온도차이 = 월[월['이화동기온'] > 월['수영동기온']]  # 6,7,8 월의 수연동 보다 높은 이화동기온 추출
# ans1 = 온도차이.shape[0] # 행 추출
# # 수연동온도 = df[(df['month'].isin([6,7,8]))& (df['hour'])]['수영동기온'].value_counts().reset_index(name='시간')
# # 더높은시간대 = 이화동온도 > 수연동온도
# # ans1 = df[(df['month'].isin([6,7,8]))& (df['hour'])]['이화동기온']\
# #                                                 .value_counts()\
# #                                                 .reset_index(name='시간')\
# #                                                 .sort_values(by='이화동기온',ascending=False)\
# #                                                 .drop_duplicates(subset='시간',keep='first')\
# #                                                 .reset_index(drop=True)\
# #                                                 .iloc[0] # 6,7,8 월의 시간대 이화동 기운 추출 이화동 기운 내림차순 정렬 시간 중복 제거 첫번째 값 추출 제일 높은 시간과 온도 추출
# ans1
# 이화동과 수영동의 최대강수량의 시간대를 각각 구하여라
df['time'] = pd.to_datetime(df['time'])
이화동강수 = df.loc[df['이화동강수'].idxmax(), 'time']
수연동강수 = df.loc[df['수영동강수'].idxmax(), 'time']
print(이화동강수,수연동강수)

2020-09-30 09:00:00 2020-07-23 12:00:00


In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churn/train.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,6842,15793491,Cherkasova,714,Germany,Male,26,3,119545.48,2,1,0,65482.94,0
1,8963,15607874,Keane,687,France,Male,38,0,144450.58,1,0,1,137276.83,0
2,7047,15737627,Rivero,589,Germany,Female,20,2,121093.29,2,1,0,3529.72,0
3,7503,15697844,Whitehouse,721,Spain,Female,32,10,0.00,1,1,0,136119.96,1
4,3439,15722404,Carpenter,445,France,Female,30,3,0.00,2,1,1,127939.19,0


In [ ]:
# 남성 이탈(Exited)이 가장 많은 국가(Geography)는 어디이고 이탈 인원은 몇명인가?
# **카드를 소유(HasCrCard ==1)하고 있으면서 활성멤버(IsActiveMember ==1) 인 고객들의 평균 나이를 소숫점이하 4자리까지 구하여라? **
# Balance 값이 중간값 이상을 가지는 고객들의 CreditScore의 표준편차를 소숫점이하 3자리까지 구하여라


In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/train.csv")
df.head()

,성별코드,연령대코드(5세단위),신장(5Cm단위),체중(5Kg단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),수축기혈압,이완기혈압,식전혈당(공복혈당),총콜레스테롤,트리글리세라이드,HDL콜레스테롤,LDL콜레스테롤,혈색소,요단백,혈청크레아티닌,(혈청지오티)AST,(혈청지오티)ALT,감마지티피,흡연상태,구강검진수검여부,치아우식증유무,치석
0,F,55,145,55,73.0,0.7,0.5,1.0,1.0,129.0,84.0,93.0,185.0,50.0,98.0,77.0,12.9,1.0,0.9,172.0,209.0,15.0,0,Y,0.0,Y
1,M,40,180,55,74.0,1.5,1.5,1.0,1.0,102.0,70.0,95.0,176.0,60.0,53.0,111.0,15.6,1.0,0.8,30.0,19.0,23.0,1,Y,0.0,Y
2,F,55,150,50,72.0,1.0,0.2,1.0,1.0,116.0,72.0,86.0,168.0,49.0,53.0,105.0,14.1,1.0,1.0,31.0,19.0,10.0,0,Y,0.0,N
3,M,40,170,85,89.0,1.0,0.8,1.0,1.0,124.0,80.0,96.0,231.0,240.0,54.0,129.0,16.7,1.0,1.0,26.0,38.0,108.0,1,Y,1.0,Y
4,F,40,155,45,62.0,0.5,1.0,1.0,1.0,127.0,85.0,93.0,261.0,94.0,71.0,171.0,12.9,1.0,0.7,20.0,11.0,13.0,0,Y,0.0,N


In [ ]:
# 수축기혈압과 이완기 혈압기 수치의 차이를 새로운 컬럼(‘혈압차’) 으로 생성하고, 연령대 코드별 각 그룹 중 ‘혈압차’ 의 분산이 5번째로 큰 연령대 코드를 구하여라
# 비만도를 나타내는 지표인 WHtR는 허리둘레 / 키로 표현한다. 일반적으로 0.58이상이면 비만으로 분류한다. 데이터중 WHtR 지표상 비만인 인원의 남/여 비율을 구하여라

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/train.csv")
df.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,188957,Female,25,1,17.0,1,< 1 Year,No,38785.0,152.0,59,0
1,275631,Male,20,1,22.0,0,< 1 Year,No,2630.0,160.0,113,0
2,329036,Female,40,1,41.0,0,1-2 Year,Yes,33165.0,124.0,188,0
3,227288,Female,28,1,30.0,1,< 1 Year,No,29038.0,152.0,233,0
4,300441,Female,50,1,28.0,0,1-2 Year,Yes,31325.0,124.0,258,0


In [ ]:
# Vehicle_Age 값이 2년 이상인 사람들만 필터링 하고 그중에서 Annual_Premium 값이 전체 데이터의 중간값 이상인 사람들을 찾고, 그들의 Vintage값의 평균을 구하여라
# vehicle_age에 따른 각 성별(gender)그룹의 Annual_Premium값의 평균을 구하여 아래 테이블과 동일하게 구현하라

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/mobile/train.csv")
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0,1


In [ ]:
# price_range 의 각 value를 그룹핑하여 각 그룹의 n_cores 의 빈도가 가장높은 value와 그 빈도수를 구하여라
# price_range 값이 3인 그룹에서 상관관계가 2번째로 높은 두 컬럼과 그 상관계수를 구하여라

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/train.csv")
df.head()

,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,55882,Male,Loyal Customer,42,Business travel,Business,473,1,1,3,1,2,5,5,5,5,5,5,3,5,3,3,0.0,satisfied
1,31906,Female,Loyal Customer,31,Business travel,Business,3271,4,4,4,4,3,4,4,3,3,5,5,3,4,3,1,5.0,satisfied
2,85481,Male,Loyal Customer,51,Personal Travel,Eco,214,5,0,5,4,3,5,3,3,4,4,4,4,4,3,2,0.0,satisfied
3,102134,Male,Loyal Customer,44,Business travel,Business,3097,1,1,3,1,3,4,5,4,4,4,4,3,4,5,0,0.0,satisfied
4,31580,Male,Loyal Customer,33,Business travel,Business,3286,1,1,1,1,4,4,4,4,2,5,2,5,1,4,0,0.0,satisfied


In [ ]:
# Arrival Delay in Minutes 컬럼이 결측치인 데이터들 중 ‘neutral or dissatisfied’ 보다 ‘satisfied’의 수가 더 높은 Class는 어디 인가?

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/waters/train.csv")
df.head()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,7.918150,214.186611,23823.492888,7.290878,341.173322,411.424483,19.585002,25.057375,4.028958,0
1,5.422446,205.266080,18542.957451,5.491963,306.702227,382.080129,10.504023,67.493450,2.911751,1
2,7.341547,187.672402,21273.457066,7.784003,NaN,332.084293,16.842334,55.019151,4.025644,0
3,9.056245,197.666301,17403.532167,7.688917,337.460176,414.766631,15.349869,63.696746,3.319354,0
4,5.039374,142.860598,40829.353167,7.271543,NaN,386.803057,16.823773,52.297113,4.957420,0


In [ ]:
# ph값은 상당히 많은 결측치를 포함한다. 결측치를 제외한 나머지 데이터들 중 사분위값 기준 하위 25%의 값들의 평균값은?

In [ ]:
import pandas as pd
train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/MedicalCost/train.csv")
train.head()

,age,sex,bmi,children,smoker,region,charges
0,64,female,39.330,0,no,northeast,14901.51670
1,47,female,27.830,0,yes,southeast,23065.42070
2,52,female,33.300,2,no,southwest,10806.83900
3,33,female,22.135,1,no,northeast,5354.07465
4,30,male,31.400,1,no,southwest,3659.34600


In [ ]:
# 흡연자와 비흡연자 각각 charges의 상위 10% 그룹의 평균의 차이는?

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/kingcountyprice//train.csv")
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,8961990160,20150413T000000,567500.0,3,2.5,2080,4556,2.0,0,0,3,8,2080,0,1999,0,98074,47.6036,-122.014,1530,5606
1,9455200205,20140604T000000,525000.0,3,2.0,1540,7800,1.0,0,0,3,8,1540,0,2004,0,98125,47.7041,-122.288,1510,7800
2,7853220670,20140918T000000,540000.0,3,2.5,2860,8935,2.0,0,0,3,8,2860,0,2004,0,98065,47.5336,-121.855,2650,6167
3,3298201170,20141110T000000,350000.0,3,1.0,940,7811,1.0,0,0,3,6,940,0,1959,0,98008,47.6195,-122.118,1180,7490
4,7972604355,20140521T000000,218000.0,3,1.0,1020,7874,1.0,0,0,3,7,1020,0,1956,0,98106,47.5175,-122.346,1290,7320


In [ ]:
# bedrooms 의 빈도가 가장 높은 값을 가지는 데이터들의 price의 상위 10%와 하위 10%값의 차이를 구하여라

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/admission/train.csv")
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,246,328,110,4,4.0,2.5,9.02,1,0.81
1,100,323,113,3,4.0,4.0,8.88,1,0.79
2,79,296,95,2,3.0,2.0,7.54,1,0.44
3,53,334,116,4,4.0,3.0,8.00,1,0.78
4,444,321,114,5,4.5,4.5,9.16,1,0.87


In [ ]:
# Serial No. 컬럼을 제외하고 ‘Chance of Admit’을 종속변수, 나머지 변수를 독립변수라 할때, 랜덤포레스트를 통해 회귀 예측을 할 떄 변수중요도 값을 출력하라 (시드값에 따라 순서는 달라질수 있음)

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/redwine/train.csv")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,5.0,0.38,0.01,1.6,0.048,26.0,60.0,0.99084,3.70,0.75,14.0,6
1,5.0,0.42,0.24,2.0,0.060,19.0,50.0,0.99170,3.72,0.74,14.0,8
2,7.1,0.36,0.30,1.6,0.080,35.0,70.0,0.99693,3.44,0.50,9.4,5
3,7.6,0.29,0.49,2.7,0.092,25.0,60.0,0.99710,3.31,0.61,10.1,6
4,7.7,0.51,0.28,2.1,0.087,23.0,54.0,0.99800,3.42,0.74,9.2,5


In [ ]:
# quality 값이 3인 그룹과  8인 데이터그룹의 각 컬럼별 독립변수의 표준편차 값의 차이를 구할때 그값이 가장 큰 컬럼명을 구하여라

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/drug/train.csv")
df.head()

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,20,F,NORMAL,NORMAL,9.281,4
1,24,M,HIGH,NORMAL,9.475,1
2,34,M,NORMAL,HIGH,22.456,0
3,17,M,NORMAL,NORMAL,10.832,4
4,72,M,LOW,HIGH,16.310,0


In [ ]:
# 남성들의 연령대별 (10살씩 구분 0~9세 10~19세 …) Na_to_K값의 평균값을 구해서 데이터 프레임으로 표현하여라

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/audit/train.csv")
df.head()

,Sector_score,LOCATION_ID,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,Risk_C,Money_Value,Score_MV,Risk_D,District_Loss,PROB,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk,Risk
0,2.72,14,4.53,0.6,2.718,87.93,0.6,52.758,92.46,5.0,0.2,1.0,85.81,0.6,51.486,2,0.2,0.4,0,0.2,0.0,4.0,108.362,0.4,0.5,21.6724,1
1,3.41,16,12.68,0.6,7.608,41.00,0.6,24.600,53.68,5.5,0.4,2.2,11.88,0.6,7.128,2,0.2,0.4,0,0.2,0.0,4.4,41.936,0.4,0.5,8.3872,1
2,2.72,11,3.11,0.6,1.866,113.97,0.6,68.382,117.08,5.5,0.4,2.2,26.64,0.6,15.984,2,0.2,0.4,0,0.2,0.0,4.4,88.832,0.4,0.5,17.7664,1
3,3.41,8,1.12,0.4,0.448,0.00,0.2,0.000,1.12,5.0,0.2,1.0,0.00,0.2,0.000,2,0.2,0.4,0,0.2,0.0,2.2,1.848,0.4,0.5,0.3696,0
4,55.57,9,1.06,0.4,0.424,0.00,0.2,0.000,1.06,5.0,0.2,1.0,0.00,0.2,0.000,2,0.2,0.4,0,0.2,0.0,2.2,1.824,0.4,0.5,0.3648,0


In [ ]:
# 데이터의 Risk 값에 따른 score_a와 score_b의 평균값을 구하여라

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/muscle/train.csv")
df.head()

,motion_0,motion_1,motion_2,motion_3,motion_4,motion_5,motion_6,motion_7,motion_8,motion_9,motion_10,motion_11,motion_12,motion_13,motion_14,motion_15,motion_16,motion_17,motion_18,motion_19,motion_20,motion_21,motion_22,motion_23,motion_24,...,motion_40,motion_41,motion_42,motion_43,motion_44,motion_45,motion_46,motion_47,motion_48,motion_49,motion_50,motion_51,motion_52,motion_53,motion_54,motion_55,motion_56,motion_57,motion_58,motion_59,motion_60,motion_61,motion_62,motion_63,pose
0,-6.0,8.0,5.0,6.0,-12.0,-27.0,5.0,24.0,-8.0,-5.0,-3.0,-2.0,5.0,13.0,-3.0,-13.0,10.0,-3.0,-1.0,-1.0,-2.0,6.0,-1.0,-1.0,-9.0,...,11.0,-2.0,1.0,1.0,-8.0,9.0,1.0,3.0,-19.0,-2.0,1.0,3.0,-8.0,-11.0,0.0,-1.0,-3.0,-6.0,-9.0,-7.0,36.0,68.0,3.0,-3.0,1
1,-32.0,-3.0,-4.0,-5.0,-1.0,16.0,0.0,-5.0,42.0,2.0,0.0,-3.0,-14.0,-23.0,-5.0,-14.0,3.0,-4.0,-4.0,-4.0,11.0,9.0,-2.0,4.0,-13.0,...,-9.0,3.0,-1.0,-1.0,13.0,19.0,0.0,3.0,-6.0,-7.0,1.0,-4.0,19.0,20.0,-3.0,-17.0,-12.0,-1.0,-1.0,0.0,-26.0,-11.0,-4.0,-9.0,1
2,35.0,-2.0,-1.0,1.0,-25.0,-16.0,-3.0,-20.0,10.0,1.0,1.0,-1.0,22.0,20.0,13.0,39.0,-5.0,10.0,0.0,8.0,2.0,6.0,2.0,-2.0,-6.0,...,14.0,3.0,7.0,12.0,6.0,-19.0,-24.0,-10.0,-12.0,-14.0,7.0,12.0,14.0,2.0,4.0,6.0,-65.0,6.0,-10.0,-31.0,-19.0,-3.0,-47.0,-17.0,0
3,-15.0,-6.0,-3.0,-5.0,27.0,37.0,-2.0,-2.0,19.0,5.0,2.0,4.0,-19.0,4.0,1.0,-1.0,24.0,2.0,0.0,0.0,-2.0,-25.0,-2.0,6.0,-1.0,...,-12.0,0.0,1.0,-2.0,-14.0,-17.0,-4.0,-13.0,5.0,1.0,1.0,4.0,16.0,4.0,-1.0,2.0,11.0,0.0,-2.0,-4.0,-7.0,15.0,3.0,14.0,1
4,60.0,3.0,0.0,-7.0,-6.0,-10.0,-6.0,7.0,-11.0,-4.0,0.0,1.0,-8.0,-4.0,-4.0,-6.0,-9.0,-2.0,-4.0,0.0,12.0,2.0,-7.0,-4.0,52.0,...,-29.0,1.0,-1.0,2.0,6.0,-2.0,-6.0,-5.0,20.0,3.0,-4.0,-25.0,4.0,-5.0,10.0,-12.0,17.0,-3.0,1.0,2.0,-2.0,20.0,18.0,10.0,0


In [ ]:
# pose값에 따른 각 motion컬럼의 중간값의 가장 큰 차이를 보이는 motion컬럼은 어디이며 그값은?

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/hyundai/train.csv")
df.head()

,model,year,price,transmission,mileage,fuelType,tax(£),mpg,engineSize
0,Tucson,2016,12795,Manual,36263,Diesel,30,61.7,1.7
1,I10,2012,3995,Manual,22089,Petrol,20,61.4,1.2
2,I30,2019,24000,Manual,555,Petrol,145,34.0,2.0
3,Tucson,2017,12995,Manual,32147,Diesel,30,61.7,1.7
4,Tucson,2018,14991,Semi-Auto,32217,Diesel,150,57.6,1.7


In [ ]:
# 정보(row수)가 가장 많은 상위 3차종의 price값의 각 평균값은?

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/diabetes/train.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,3,102,74,0,0,29.5,0.121,32,0
1,2,144,58,33,135,31.6,0.422,25,1
2,5,136,82,0,0,0.0,0.640,69,0
3,13,145,82,19,110,22.2,0.245,57,0
4,1,117,60,23,106,33.8,0.466,27,0


In [ ]:
# Outcome 값에 따른 각 그룹의  각 컬럼의 평균 차이를 구하여라

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/nflx/NFLX.csv")
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2002-05-23,1.156429,1.242857,1.145714,1.196429,1.196429,104790000
1,2002-05-24,1.214286,1.225000,1.197143,1.210000,1.210000,11104800
2,2002-05-28,1.213571,1.232143,1.157143,1.157143,1.157143,6609400
3,2002-05-29,1.164286,1.164286,1.085714,1.103571,1.103571,6757800
4,2002-05-30,1.107857,1.107857,1.071429,1.071429,1.071429,10154200


In [ ]:
# 매년 5월달의 open가격의 평균값을 데이터 프레임으로 표현하라

In [ ]:
pd.set_option('display.max_columns',50)
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/nba/nba.csv",encoding='latin',sep=';')
df.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Precious Achiuwa,C,22,TOR,73,28,23.6,3.6,8.3,0.439,0.8,2.1,0.359,2.9,6.1,0.468,0.486,1.1,1.8,0.595,2.0,4.5,6.5,1.1,0.5,0.6,1.2,2.1,9.1
1,2,Steven Adams,C,28,MEM,76,75,26.3,2.8,5.1,0.547,0.0,0.0,0.000,2.8,5.0,0.548,0.547,1.4,2.6,0.543,4.6,5.4,10.0,3.4,0.9,0.8,1.5,2.0,6.9
2,3,Bam Adebayo,C,24,MIA,56,56,32.6,7.3,13.0,0.557,0.0,0.1,0.000,7.3,12.9,0.562,0.557,4.6,6.1,0.753,2.4,7.6,10.1,3.4,1.4,0.8,2.6,3.1,19.1
3,4,Santi Aldama,PF,21,MEM,32,0,11.3,1.7,4.1,0.402,0.2,1.5,0.125,1.5,2.6,0.560,0.424,0.6,1.0,0.625,1.0,1.7,2.7,0.7,0.2,0.3,0.5,1.1,4.1
4,5,LaMarcus Aldridge,C,36,BRK,47,12,22.3,5.4,9.7,0.550,0.3,1.0,0.304,5.1,8.8,0.578,0.566,1.9,2.2,0.873,1.6,3.9,5.5,0.9,0.3,1.0,0.9,1.7,12.9


In [ ]:
# Tm 컬럼은 각 팀의 이름을 의미한다. TOR팀의 평균나이를 소수 4째 자리까지 구하여라
# Pos 컬럼은 포지션을 의미한다. 전체 선수 중 최소나이대의 선수들을 필터하고 그들 중 가장 많은 포지션은 무엇인지 확인하라
# 선수들의 이름은 first_name+ 공백 + last_name으로 이루어져 있다. 가장 많은 first_name은 무엇이며 몇 회 발생하는지 확인하라
# PTS컬럼은 경기당 평균득점수 이다. 각포지션별로 경기당 평균득점수의 평균을 구하여라
# PTS컬럼은 경기당 평균득점수 이다. 각포지션별로 경기당 평균득점수의 평균을 구하여라
# G컬럼은 참여한 경기의 숫자이다. 각 팀별로 가장 높은 경기참여 수를 가진 선수들의 경기 참여 숫자의 평균을 구하여라
# Tm의 값이 MIA이며 Pos는 C또는 PF인 선수의 MP값의 평균은?
# 전체 데이터중 G의 평균값의 1.5배 이상인 데이터들만 추출했을때 3P값의 평균은?
# Age의 평균 이상인 그룹과 평균 미만인 그룹간의 G값의 평균의 차이는?
# 평균나이가 가장 젊은 팀은 어디인가
# Pos그룹별 평균 MP값을 구하여라